<a href="https://colab.research.google.com/github/erinmr/ATLS-5519/blob/main/Day_3_ER_Submitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 3 - June 2, 2022
We covered problems 7 and 8. Problem 9 was homework. 

Problem 7 was to count syllables using the CMU_dict and creating an exception dictionary with missing. 

Problem 8 was modified to create a line of poetry with a user-specified number of syllables using the Markov model 1 

Problem 9 was to test a set of random words from diction 2of4brif.txt with the syllable counter in Problem 7. 

This first section sets up the CMU dict, which is a phoentic dictionary and missing words file to create our corpus. 

In [ ]:

"""Find words in haiku corpus missing from cmudict & build exceptions dict."""
import nltk
nltk.download('cmudict')
  

import sys
from string import punctuation
import pprint
import json
from nltk.corpus import cmudict

cmudict = cmudict.dict()  # Carnegie Mellon University Pronouncing Dictionary

def load_haiku(filename):
    """Open and return training corpus of haiku as a set."""
    with open(filename) as in_file:
        haiku = set(in_file.read().replace('-', ' ').split())
        return haiku

def cmudict_missing(word_set):
    """Find and return words in word set missing from cmudict."""
    exceptions = set()
    for word in word_set:
        word = word.lower().strip(punctuation)
        if word.endswith("'s") or word.endswith("’s"):
            word = word[:-2]
        if word not in cmudict:
            exceptions.add(word)
    print("\nexceptions:")
    print(*exceptions, sep='\n')
    print("\nNumber of unique words in haiku corpus = {}".format(len(word_set)))
    print("Number of words in corpus not in cmudict = {}"
          .format(len(exceptions)))
    membership = (1 - (len(exceptions) / len(word_set))) * 100
    print("cmudict membership = {:.1f}{}".format(membership, '%'))
    return exceptions

def make_exceptions_dict(exceptions_set):
    """Return dictionary of words and syllable counts from set of words."""
    missing_words = {}
    print("Input # syllables in word. Mistakes can be corrected at end. \n")
    for word in exceptions_set:
        while True:
            num_sylls = input("Enter number syllables in {}: ".format(word))
            if num_sylls.isdigit():
                break
            else:
                print("                   Not a valid answer!", file=sys.stderr)                    
        missing_words[word] = int(num_sylls)              
    print()
    pprint.pprint(missing_words, width=1)

    print("\nMake Changes to Dictionary Before Saving?")
    print("""
    0 - Exit & Save
    1 - Add a Word or Change a Syllable Count 
    2 - Remove a Word
    """)

    while True:
        choice = input("\nEnter choice: ")   
        if choice == '0':
            break
        elif choice == '1':
            word = input("\nWord to add or change: ")
            missing_words[word] = int(input("Enter number syllables in {}: "
                                            .format(word)))
        elif choice == '2':
            word = input("\nEnter word to delete: ")
            missing_words.pop(word, None)
            
    print("\nNew words or syllable changes:")
    pprint.pprint(missing_words, width=1)

    return missing_words

def save_exceptions(missing_words):
    """Save exceptions dictionary as json file."""
    json_string = json.dumps(missing_words)
    f = open('missing_words.json', 'w')
    f.write(json_string)
    f.close()
    print("\nFile saved as missing_words.json")


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


Note: Only run this if you need to build the missing_words.json file 

In [ ]:
haiku = load_haiku('train.txt')
exceptions = cmudict_missing(haiku)
build_dict = input("\nManually build an exceptions dictionary (y/n)? \n")
if build_dict.lower() == 'n':
    sys.exit()
else:
    missing_words_dict = make_exceptions_dict(exceptions)
    save_exceptions(missing_words_dict)

# Problem 7: Syllable counter 

Objective: Write a Python program that counts the number of syllables in an English word or phrase.

Things to consider: 
- List of vowel sounds or instead look for numbers at the end of phenomes 


Pseudocode:
- count vowels by the string with a number approach
 - creat a list of all vowel string sounds (ALT - look for numbers at the end 
 - allow user to input a word or phrase 
 - Load CMU Dictionary
 - Load missing_words.json
 - Loop on input determine if this is a word or phrase 
 - if phrase separate each word at the space 
 - if word no spaces found
 - for each word
    - count the number of times the vowels occur in the word
      - Check in missing_words and add to a counter
      - else look in dictionary file for the word(s)
        - look for numbers at the end of phenomes to indicate vowles (and syllabuls) and add to counter  
   
 - print word or phrase and counter total number of syllables 
 - Similar to approach yesterday to count letters 


In [ ]:
import sys
from string import punctuation
import json
from nltk.corpus import cmudict

cmudict = cmudict.dict()

with open('missing_words.json') as f: 
  missing_words = json.load(f)
 # print(missing_words)

def count_syllables(words): #some user input (word or phrase)
  num_sylls = 0 
 
  for word in words.lower().split(): 
   # print(word)

    if word in missing_words: 
      num_sylls += missing_words[word]
      #print(word)
    
    elif word in cmudict.keys():
      #print(word) 
      for phonemes in cmudict[word][0]:
        for phoneme in phonemes:
          #print(phoneme)
          if phoneme.isdigit():
            num_sylls += 1
    else: 
      a_missing_word = input("We don't have that word. How many syllables does it have?")
      a_missing_word = int(a_missing_word)
      num_sylls = num_sylls + a_missing_word

  return num_sylls


word = input("Enter a word or phrase")
num_syllables = count_syllables(word)
print(num_syllables)

Enter a word or phrasecat
1


# Problem 8 - Poetry Line Generator 


Objective: Write a program that generates haiku using Markov chain analysis. Allow the user to modify the haiku by independently regenerating lines two and three.

standing still at dusk
listen... in far distances 
the song of frogglings

x:y
'standing': 'still' 

'x y': 'z' 
'standing still': 'at'
'still at': 'dusk' 

Pseudocode:
- load training file 
- Clean up corpus for punctuation 
- build models 
  - Markov 1 (first word predicts second word)
  - Markov 2 (first two words predict third word)
- line creation function that leverages Markov 1 and 2
- user enters number of syllables for the phrase (target sylls) 
- Identify word 1 by picking random word or seeding word or phrase from user
- count syllables of word 1 = num sylls
- set phrase = to word 1
- while num sylls < target sylls find new word and count word sylls. if num sylls + new sylls <= add word to phrase and set word = new word 
- Print phrase 



In [47]:
import sys
from string import punctuation
import json
from nltk.corpus import cmudict

cmudict = cmudict.dict()

with open('missing_words.json') as f: 
  missing_words = json.load(f)
 # print(missing_words)

def count_syllables(words): #some user input (word or phrase)
  num_sylls = 0 
 
  for word in words.lower().split(): 

    if word in missing_words: 
      num_sylls += missing_words[word]
    
    elif word in cmudict.keys():
      #print(word) 
      for phonemes in cmudict[word][0]:
        for phoneme in phonemes:
          #print(phoneme)
          if phoneme.isdigit():
            num_sylls += 1
    else: 
      a_missing_word = input(f"We don't have the word {word}. How many syllables does it have?")
      a_missing_word = int(a_missing_word)
      num_sylls = num_sylls + a_missing_word

  return num_sylls

import random
from collections import defaultdict

with open('train.txt') as f:
  raw_haiku = f.read()
  
#cleaning up the corpus for weird punctuation 
corpus = raw_haiku.replace('\n', ' ')
corpus = corpus.replace('!', '')
corpus = corpus.replace(',', '')
corpus = corpus.replace('-', ' ')
corpus = corpus.replace('?', '')
corpus = corpus.replace("'s", '')
corpus = corpus.replace(':', '')
corpus = corpus.replace('"', '')
corpus = corpus.replace('.', '')
corpus = corpus.split()

#create Markov Model for one word
def map_word_to_word(corpus):
  limit = len(corpus)-1
  dict1_to_1 = defaultdict(list)
  
  for index, word in enumerate(corpus):
    if index < limit:
      suffix = corpus[index+1]
      dict1_to_1[word].append(suffix)

  return(dict1_to_1)

#create Markov Model for two word phrase
def map_2_words_to_word(corpus):
  limit = len(corpus)-2
  dict2_to_1 = defaultdict(list)
  
  for index, word in enumerate(corpus):
    if index < limit:
      key = word + ' ' + corpus[index+1]
      suffix = corpus[index+2]
      #suffix = corpus[index+1]
      dict2_to_1[key].append(suffix)

  return(dict2_to_1)

# pull a random word from the corpus 
def random_word(corpus):
  word = random.choice(corpus)
  num_sylls = count_syllables(word)
  return(word,num_sylls)

# gather the next word after the current word
def word_after(prefix, suffix_map, current_syls, target_syls):
  accepted_words = []
  suffixes = suffix_map.get(prefix)
  
  for candidate in suffixes:
    num_syls = count_syllables(candidate)
    if current_syls + num_syls <= target_syls:
      accepted_words.append(candidate)              # create a list of words with acceptable syllable counts

  final_word = random.choice(accepted_words)        # choose a random word from the accepted list
 
  return(final_word)

target_syllables = int(input('Enter number of syllables for phrase (5 or 7):'))
word, num_syllables = random_word(corpus)
phrase = word

while num_syllables < target_syllables:
    new_word = word_after(word, map_word_to_word(corpus), num_syllables, target_syllables)
    new_syllables = count_syllables(new_word)
    if num_syllables + new_syllables <= target_syllables:       # if nbew word is within
        phrase = phrase + " " + new_word
        num_syllables += new_syllables
        word = new_word

print(f'Your poetry line is: {phrase} ({count_syllables(phrase)} syllables)')


Enter number of syllables for phrase (5 or 7):5
Your poetry line is: do your very bones (5 syllables)


# Problem 9 - Test Syllable Counter 
With user defined number of random words 

Pseudocode:
1. load in the new corpus - 2of4brif.txt
2. Ask the user how many words to test 
2. strip the word list to remove any leading/trailing spaces 
3. set j = 0
4. While j is less than or equal to the target num of words set by the user
- generate a random number 
- use that as the index in the corpus to find a word 
- run the count syllable function
- if the word is not found enter the number of syllables 




In [ ]:
f = open("/content/2of4brif.txt", 'r')  # open the dictionary file 

target_num_words = int(input('Enter number of words to test:'))  # get target number of words to test 
words_list = f.readlines()

for i in range(len(words_list)): 
  words_list[i] = words_list[i].strip()

j=0

while j <= target_num_words-1:
  random_number = random.randint(0,len(words_list)-1)
  word = words_list[random_number]
  syllables = count_syllables(word)
  print(f'the word is {word} and it has {syllables} syllables')
  j +=1 
  print(j)



 
  

#print(random_number1)



Enter number of words to test:18
We don't have the word teatimes. How many syllables does it have?1
the word is teatimes and it has 1 syllables
1
We don't have the word lynchpin. How many syllables does it have?2
the word is lynchpin and it has 2 syllables
2
We don't have the word vaccinates. How many syllables does it have?3
the word is vaccinates and it has 3 syllables
3
the word is initiative and it has 4 syllables
4
the word is piranha and it has 3 syllables
5
We don't have the word backcloth. How many syllables does it have?2
the word is backcloth and it has 2 syllables
6
the word is readying and it has 3 syllables
7
the word is snippet and it has 2 syllables
8
the word is sickle and it has 2 syllables
9
the word is way and it has 1 syllables
10
the word is cites and it has 1 syllables
11
We don't have the word roguishly. How many syllables does it have?3
the word is roguishly and it has 3 syllables
12
the word is bottoms and it has 2 syllables
13
the word is incorrect and it has 

✨ made it through the first week! 